In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
# sys.path.insert(0, "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages")
import argparse
import numpy as np
from random import uniform as random
import random as rand
from math import pi as PI
from math import cos, sin, atan2, atan, tanh, fmod, exp, isclose
from math import hypot as mag
import json
from itertools import takewhile
from namespace import Namespace
import tensorflow as tf

In [3]:
dir(tf.version)
tf.version.VERSION

'2.0.0-rc0'

In [4]:
tf.test.is_built_with_rocm()

True

In [5]:
tf.test.is_gpu_available()

True

In [ ]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:

HEADLESS = True
FRAMESKIP = 5
should_write_training_data = False
should_read_training_data = False
BATCHES = 10
N_PLAYERS = 16
scale = 20
should_draw_vectors = False
debug = False


# import tensorflow as tf


HALF_PI = PI / 2
TAU = PI * 2

g = 9.80665
drag_flat = 0.07
drag_narrow = 0.038
mass = 100
influence = 900

MUTATION_EFFECT = 0.20
MUTATION_CHANCE = 0.20
timedelta = 0.1

N_PARAMS = 11
TTL = 1000
OFFSET = (100, 100)
RANDOM_LOWER_BOUND = 15000
RANDOM_UPPER_BOUND = 15000
PARAM_LOWER_BOUND = -1
PARAM_UPPER_BOUND = 1
RANDOM_INITIAL = 15000
FITNESS_HYPERPARAMETER_WIDTH=10000
FITNESS_HYPERPARAMETER_HEIGHT=30000
resources = Namespace()

offset = 0
text_to_render = []
longest_width = 0


def render_text(text, color=(0, 0, 0)):
    global offset
    global text_to_render
    global longest_width
    if HEADLESS:
        return
    if not isinstance(text, str):
        text = str(text)
    font = resources.font
    size = font.size(text)
    h = size[1]
    size = pygame.Rect((0, offset), size)
    offset += h
    if size[0] > longest_width:
        longest_width = size[0]
    text = font.render(text, False, color)
    text_to_render.append((text, size))


def clear_text_buffer():
    global offset
    global text_to_render
    global longest_width
    longest_width = 0
    offset = 0
    text_to_render = []


def sign(x):
    return 1 if x >= 0 else -1


def vadd(l1, l2):
    return l1[0] + l2[0], l1[1] + l2[1]


def vsub(l1, l2):
    return l1[0] - l2[0], l1[1] - l2[1]


def out_of_bounds(player):
    return player[0] < -100 or player[1] > player[6]

def direction(player):
    return atan2(player[3], player[2]) if player[2] != 0 else (PI+HALF_PI)

def AoA(player):
    # return atan2(player[3], player[2])
    return np.mod(direction(player) - player[4], TAU)

def tangent(player):
    return ((atan2(-player[2],player[3]) + (PI if player[3] > 0 else 0)) if player[3] != 0 else HALF_PI) % TAU


def pmag(player):
    return mag(player[2], player[3])


def lift_force(player):
    # angle = (self.theta - self.direction) % TAU

    y = 0.7 * 1.225 * 0.75 / (2*mass)
    # normal lift
    _AoA = AoA(player)
    _tan = tangent(player)
    mul = 50*y * pmag(player)**2 * cos(_AoA) * sin(_AoA)
    return [mul*cos(_tan), mul*sin(_tan)]

def simulate(player):
    L = lift_force(player)
    _mag = mag(player[2], player[3])
    # f = drag_narrow + (drag_flat-drag_narrow) * abs(sin(self.AoA))
    f = drag_narrow
    player[2] += (L[0] - f * _mag * player[2])/mass
    player[3] += (L[1] - g * mass - f * _mag * player[3])/mass
    player[0] += player[2] * timedelta
    player[1] -= player[3] * timedelta
    player[7] += timedelta

def transform_pos(x=0, y=0):
    """returns the screen coordinates to draw pos"""
    return vadd(OFFSET, (int(x / scale), int(y / scale)))


def reverse_transform_position(x, y):
    """returns the approximate real coordinates that correspond to screen coordinates"""
    return vsub([e*scale for e in [x,y]], OFFSET)

def player_fitness_formula(player):
    return fitness_formula(player[0], player[1], player[5], player[6], player[7])

def fitness_formula(x, y, tx, ty, time):
    return FITNESS_HYPERPARAMETER_HEIGHT * exp(-(mag(tx-x, ty-y) / FITNESS_HYPERPARAMETER_WIDTH) ** 2) - 2*time

assert fitness_formula(0,0,0,0,0) == 30000

def prepare_bg(tx, ty, SIZE, fitness_formula):
    surface = pygame.Surface((SIZE)).convert_alpha()
    for y in range(0, SIZE[1], 10):
        for x in range(0, SIZE[0], 10):
            nx, ny = reverse_transform_position(x, y)
            v = fitness_formula(nx, ny, tx, ty, 0)
            v = 1+exp(-1) - exp(-v/30000)
            # print(pos, v)
            surface.fill(pygame.Color(*[int(e*255) for e in colorsys.hls_to_rgb(v, 0.5, 0.5)]), (x,y, 10,10))
    return surface


In [ ]:
def player_fitness_formula(player):
    return fitness_formula(player[0], player[1], player[5], player[6], player[7])

def loss(x, y, tx, ty, time):
    return FITNESS_HYPERPARAMETER_HEIGHT - FITNESS_HYPERPARAMETER_HEIGHT * exp(-(mag(tx-x, ty-y) / FITNESS_HYPERPARAMETER_WIDTH) ** 2) - 2*time


In [ ]:

ONE_BRAIN = None
def construct_brain():
    global ONE_BRAIN
    if ONE_BRAIN is None:
        ONE_BRAIN = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(10,))])
    return ONE_BRAIN


def mutation(brain):
    for i, param in enumerate(brain):
        if random(0, 1) < MUTATION_CHANCE:
            # print("mutation occurred!")
            # brain.brain.params[i] = random(-MUTATION_EFFECT, MUTATION_EFFECT) + param
            brain[i] = rand.gauss(0, 2 * MUTATION_EFFECT) + param


PROPORTIONS = [0.25, 0.25, 0.6, 0.15]
def selection_crossover_and_breeding(fitnesses, brain):
    mutation(brain)
    return brain

def remap(variables):
    return [
        variables[0], # x
        variables[1], # y
        pmag(variables), # mag
        direction(variables), # dir
        variables[4], # theta
        mag(variables[5]-variables[0], variables[6]-variables[1]), # distance to target
        atan2(variables[6]-variables[1], variables[5]-variables[0]), # angle to target
        variables[5]-variables[0], # horizontal distance to target
        variables[6]-variables[1], # vertical distance to target
        variables[7] # time
    ]




def update(player, prediction):
    player[4] = prediction

def empty_player_constructor(target):
    return tf.constant([
        0, # x
        0, # y
        0, # vx
        0, # vy
        0, # theta
        target[0], # target x
        target[1], # target y
        0, # time
        True, # alive
        0 # fitness
    ])


def reset(player):
    player[0:5] = [0,0,0,0,0]
    player[7] = 0
    player[-2] = True
    player[-1] = 0


def construct_players(DEST, filename=None):
#     players = []
#     brain = construct_brain()
#     print("constructing brain from params")
#     for i in range(N_PLAYERS):
#         players.append(construct_player(DEST))
#         update(players[-1], fmod(i*0.2314, TAU))
#     players = tf.TensorArray(empty_player_constructor(DEST), dtype=tf.dtypes.float32, size=N_PLAYERS)
    players = [empty_player_constructor(DEST) for _ in range(N_PLAYERS)]
    return players

def generate_data(tracks):
#     new_tracks = np.array(tracks)
    print(tracks[:,2,:].shape)
    truncated_tracks = [[new_tracks[k,pi,:] for k in range(new_tracks[:,pi,:].shape[0]) if new_tracks[k,pi,-2] == 1] for pi in range(new_tracks.shape[1])]
    xs = [remap(e) for track in truncated_tracks for e in track]
    ys = [new_tracks[-1][i][-1] for i in range(new_tracks.shape[1]) for k in range(new_tracks[:,i,:].shape[0])]
#     print(len(truncated_tracks))
#     print(list(len(e) for e in truncated_tracks))
#     print(len(list(len(e) for e in truncated_tracks[1])))
#     print(list(list(track[:][-1]) for track in truncated_tracks))
    return xs, ys

In [ ]:

optimizer = tf.keras.optimizers.SGD()
FLOOR = 10000

DEST = RANDOM_INITIAL, FLOOR



brain = construct_brain()

i = 0
global_step = tf.Variable(0)
print(f"starting batch {i} of {BATCHES}")
frame = 0

alive_players_count = N_PLAYERS
S = 0

best_fitness = float("inf")

with tf.GradientTape() as gradient:
    players = construct_players(DEST)
    tracks = tf.TensorArray(dtype=tf.dtypes.float32, size=16)
    while True:
        for player in players:
            if not player[-2]:
                continue
            simulate(player)

            if player[-2] and out_of_bounds(player):
                player[-2] = False # assign alive status
                player[-1] = player_fitness_formula(player) # assign fitness
                print(player[-1])
                # print(player[-1])
                alive_players_count -= 1
                continue
            if player[-2] and player[7] > TTL:
                player[-2] = False # assign alive status
                player[-1] = player_fitness_formula(player) # assign fitness
                print(player[-1])
                # print(player[-1])
                alive_players_count -= 1
                continue
        tracks.append(np.array([player.copy() for player in players]))
        if alive_players_count == 0:
            break
        states = np.array([remap(player) for player in players if player[-2]])
        predictions = brain.predict(states)
        for i, player in enumerate([player for player in players if player[-2]]):
            player[4] = predictions[i][0]
        render_text(f"alive players = {alive_players_count}")

        clear_text_buffer()

    xs, ys = generate_data(tracks)
    loss = sum(ys)
    print(loss)
    best_fitness = max(*[e[-1] for e in players])
    # print("new target = " + str(new_target))
    # print(f"batch {i} of {BATCHES} = {round(100*i/BATCHES,2)}% done")
    # print(f"best fitness was {best_fitness}")
    

grads = gradient.gradient(loss, brain.trainable_variables)
print("performing batch gradient application step")
print(grads)
optimizer.apply_gradients(zip(grads, brain.trainable_variables), global_step)
# for player in players:
#     player[5:7] = new_target
# assert len(set(id(e) for e in brains)) == len(brains)
# print("resetting players")
# print("player 0 before reset", players[0])
# for player in players:
#     reset(player)
# print("player 0 after reset", players[0])
# print("done resetting players")
new_target = random(RANDOM_LOWER_BOUND, RANDOM_UPPER_BOUND), FLOOR
DEST = new_target